<a href="https://colab.research.google.com/github/LorenzoCorbinelli/MLSA-project/blob/main/Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment setup

In [ ]:
!pip install transformers datasets

In [ ]:
import os
import pandas as pd
import torch
import numpy as np
from transformers import RobertaTokenizer, RobertaConfig, AutoModel, DataCollatorForLanguageModeling, RobertaForMaskedLM
from datasets import Dataset as ds
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torch.optim import Adam

# Load the dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Download the dataset
!wget http://files.srl.inf.ethz.ch/data/py150_files.tar.gz

In [ ]:
!tar -xvzf py150_files.tar.gz # unzip the folder containing the dataset

In [ ]:
!tar -xvzf data.tar.gz # unzip the dataset

After unzipping the data.tar.gz folder, we retrieve the python files, removing all the comments because they are not usefull for our purposes

Extract all the source code of the python files into a dataframe. One file is reported into a single line, including the reference of the file itself

In [ ]:
def load_python_files(root_dir):
    """Loads all Python files in a directory recursively into a DataFrame."""
    all_files = []
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename.endswith(".py"):  # take only python files
                filepath = os.path.join(dirpath, filename)
                try:
                    with open(filepath, 'r', encoding='utf-8') as f:
                        content = f.readlines() # read the file line by line
                        filtered_content = []
                        inside_triple_quotes = False
                        for line in content:
                            stripped_line = line.strip()
                            if '"""' in stripped_line:
                                if stripped_line.count('"""') == 2:
                                    continue  # Ignore lines with both opening and closing triple quotes
                                inside_triple_quotes = not inside_triple_quotes
                                continue
                            if inside_triple_quotes or stripped_line.startswith('#'): # ignore line that starts with # or check if I am inside a multiline comment
                                continue
                            filtered_content.append(line)
                        all_files.append({'filepath': filepath, 'snippet_of_code': ''.join(filtered_content)})
                except Exception as e:
                    print(f"Error reading file {filepath}: {e}")
    return pd.DataFrame(all_files)


In [ ]:
data_dir = "/content/data"
df_python_files = load_python_files(data_dir)

In [ ]:
df_train = df_python_files["snippet_of_code"].iloc[0:2000] # take only the first 2000 snippets for training
df_eval = df_python_files["snippet_of_code"].iloc[2000:3000] # take 1000 snippets for evaluation

# Tokenization

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('microsoft/codebert-base-mlm')
# tokenizer arguments to properly handle the tokenization of the snippets
tokenizer_kwargs = dict(truncation=True, padding=True, max_length=500, add_special_tokens=True)

In [ ]:
'''
We have used DataCollatorForLanguageModeling in order to tokenize the dataset and mask some tokens
It will automatically generate the labels for the masked tokens.
For the tokens not masked the label will be -100
'''
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [ ]:
def tokenize_dataset(dataset, tokenizer, **kwargs):
    token_ids = tokenizer(dataset, return_tensors='pt', **kwargs)
    return token_ids

In [ ]:
train_tensor_dataset = tokenize_dataset(df_train.to_list(), tokenizer, **tokenizer_kwargs)
eval_tensor_dataset = tokenize_dataset(df_eval.to_list(), tokenizer, **tokenizer_kwargs)

datasetTrain = ds.from_dict(train_tensor_dataset)
datasetEval = ds.from_dict(eval_tensor_dataset)

datasetTrain.set_format(type='torch', columns=['input_ids', 'attention_mask'])
datasetEval.set_format(type='torch', columns=['input_ids', 'attention_mask'])

generator = torch.Generator()
train_loader = DataLoader(datasetTrain, batch_size=4, shuffle=True, generator=generator, collate_fn=data_collator)
eval_loader = DataLoader(datasetEval, batch_size=4, collate_fn=data_collator)

In [ ]:
model = RobertaForMaskedLM.from_pretrained('microsoft/codebert-base-mlm')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = Adam(model.parameters(), lr=1e-5)

# Training loop
for epoch in range(2):
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        inputs = {key: val.to(device) for key, val in batch.items() if key in tokenizer.model_input_names}
        inputs['labels'] = batch['labels'].to(device)
        outputs = model(**inputs)
        loss = outputs.loss
        print(loss)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch + 1} completed")


In [ ]:
# Evaluation loop

model.eval()
with torch.no_grad():
    for batch in eval_loader:
        input = {key: val.to(device) for key, val in batch.items()}
        outputs = model(**input)
        print(outputs.loss)

In [ ]:
# save the model
directory = 'path_to_the_model_directory'

model.save_pretrained(directory)
tokenizer.save_pretrained(directory)